# Plotly Dashboard Project

In [1]:
import pandas as pd
import numpy as np

import plotly.graph_objects as go
import plotly.express as px

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

import dash_daq as daq

import requests
import datetime
import json


list(pio.templates)

NameError: name 'pio' is not defined

In [47]:
print("Dash       : ", dash.__version__)
print("Plotly     : ", plotly.__version__)
print("Pandas     : ", pd.__version__)
print("Numpy      : ", np.__version__)
print("Ipywidgets : ", widgets.__version__)

Dash       :  1.9.1
Plotly     :  4.5.2
Pandas     :  0.25.1
Numpy      :  1.17.1
Ipywidgets :  7.5.1


In [48]:
csi_df = pd.read_csv("csi_and_sector_data.csv", index_col=0, parse_dates=True)
csi_df.head()

,csi1,csi2,XLY,XLK,XLI,XLP,XLU,XLV,XSD,XLF,XLE,XRT,XLB
date,,,,,,,,,,,,,
2017-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-01-09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
quadrant_df = pd.read_csv('quadrant_data.csv', index_col=0, parse_dates=True)
print(quadrant_df.shape)
quadrant_df.head()

(20, 105)


,JNK_value1,JNK_value2,JNK_RoC,JNK_RoC_qqe,JNK_gmean,LQD_value1,LQD_value2,LQD_RoC,LQD_RoC_qqe,LQD_gmean,...,RPV_value1,RPV_value2,RPV_RoC,RPV_RoC_qqe,RPV_gmean,XLG_value1,XLG_value2,XLG_RoC,XLG_RoC_qqe,XLG_gmean
2019-12-03,-12.823569,-12.409707,0.222793,91.166946,89.312837,-6.499521,-6.275703,0.912584,96.814048,95.717150,...,0.542598,1.003464,-1.410548,93.334903,94.625817,5.037974,4.750255,0.138202,105.928390,105.240935
2019-12-04,-12.836278,-12.409707,0.418004,93.042573,90.515868,-6.435924,-6.275703,0.589639,97.981984,96.319043,...,0.148077,0.608414,-1.272058,92.162693,93.811728,5.062690,4.776323,0.124271,106.064376,105.814991
2019-12-05,-12.867455,-12.409707,0.400183,94.872696,92.085342,-6.423959,-6.275703,0.267739,98.827099,97.205412,...,-0.218677,0.241432,-0.583059,91.804359,93.005195,5.095738,4.810965,0.048403,105.620492,105.802976
2019-12-06,-13.052652,-12.576311,-0.073941,96.072164,93.770122,-6.564424,-6.275703,-0.243824,99.214827,98.205160,...,-0.535960,-0.076168,-0.495639,91.597540,92.222436,5.164801,4.882082,0.057592,105.290266,105.725459
2019-12-09,-13.105844,-12.629854,0.081538,97.588515,95.379449,-6.633593,-6.275703,-0.113408,99.726819,98.935626,...,-0.764576,-0.305191,-0.245831,91.781572,91.836313,5.174158,4.893882,-0.144538,103.390034,105.086314


In [50]:
def create_csi_chart():
    fig = go.Figure()

    scat1 =  go.Scatter(x=csi_df.index, y=csi_df["csi1"], name="CSI",
                        line = dict(color="dodgerblue"),
                        hovertemplate="Ticker: CSI<br>"+\
                                      "X: %{x}<br>Y: %{y: .2f}"
                   )


    scat2 =  go.Scatter(x=csi_df.index, y=csi_df["csi2"], name="CSI Index Lag",
                        line = dict(color='purple', dash='dot'),
                        hovertemplate="Ticker: CSI Index Lag<br>"+\
                                      "X: %{x}<br>Y: %{y: .2f}"
                    )

    charts= [scat1, scat2]
    fig = go.Figure(data=charts)
    fig.update_layout(title_text="Cyclical Strength Index", 
                      xaxis_title='Date Range',
                      yaxis_title='Index',
                      legend_title_text="Ticker",
                      height=600,
                      template="plotly_dark"
                      )

    fig.update_xaxes(rangeslider_visible=True,
                     rangeselector=dict(
                                     buttons=[
                                     dict(count=1, label="1m", step="month", stepmode="backward"),
                                     dict(count=3, label="3m", step="month", stepmode="backward"),
                                     dict(count=6, label="6m", step="month", stepmode="backward"),
                                     dict(count=1, label="YTD", step="year", stepmode="todate"),
                                     dict(count=1, label="1y", step="year", stepmode="backward"),
                                     dict(step="all")
                                    ],
                                     font = dict(size=12, color='#000000')
                                )
                    )

    return fig

In [51]:
%time create_csi_chart()

CPU times: user 312 ms, sys: 0 ns, total: 312 ms
Wall time: 311 ms


In [52]:
#for col in csi_df.columns[2:3]:
#    scat3 = go.Scatter(x=csi_df.index, y=csi_df[col], name=col,
#                   line = dict(color="tomato", dash='dot'),
#                   text=[col]*len(csi_df) ,
#                   hovertemplate="Ticker: %{text}<br>"+\
#                                 "X: %{x}<br>Y: %{y: .2f}")
#    fig.add_trace(scat3)

#fig.show()

In [53]:
ticker_dict = {
    'JNK' : 'Junk Bonds',
    'LQD' : 'Corporate Bonds',
    'CWB' : 'Convertible Securities',
    'ITA' : 'Aerospace & Defense',
    'HYG' : 'High Yield Bonds',
    'PBS' : 'Dynamic Media',
    'PSP' : 'Private Equity',
    'PRF' : 'FTSE 1000',
    'IWS' : 'MidCap Value',
    'RSP' : 'Eql Weight SP500',
    'XLB' : 'Materials',
    'IWP' : 'MidCap Growth',
    'IAI' : 'Broker Dealers',
    'IWR' : 'MidCap Index',
    'SPXL' : 'Levered 3x SP500',
    'ITOT' : 'SP1500',
    'IHI' : 'Medical Devices',
    'PRFZ' : 'Preferred Stock',
    'JKE' : 'LargeCap Growth',
    'RPV' : 'SP Pure Value',
    'XLG' : 'Top-50 SP500'
}

In [54]:
fig = go.Figure()

xs, ys = [], []

for ticker in list(ticker_dict.keys()):
    x = quadrant_df[ticker+'_value1'].values[0]
    y = quadrant_df[ticker+'_gmean'].values[0]
    xs.append(x)
    ys.append(y)
    fig.add_trace(go.Scatter(x = [x],y = [y], 
                             customdata=[(ticker, ticker_dict[ticker])],
                             hovertemplate="Ticker: %{customdata[0]}<br>"+\
                                           "Name: %{customdata[1]}<br>"+\
                                           "X: %{x: .2f}<br>Y: %{y: .2f}",
                             name=ticker,
                             marker=dict(size=8)))

fig.add_shape(
                type="rect",
                x0=min(xs)-5, 
                y0=min(ys)-5,
                x1=0,
                y1=100,
                fillcolor="red", opacity=0.3,
                line=dict(width=0),
)

fig.add_shape(
                type="rect",
                x0=0, 
                y0=100,
                x1=max(xs)+5,
                y1=max(ys)+5,
                fillcolor="green", opacity=0.3,
                line=dict(width=0),
)

fig.add_shape(
                type="rect",
                x0=min(xs)-5, 
                y0=100,
                x1=0,
                y1=max(ys)+5,
                fillcolor="yellow", opacity=0.3,
                line=dict(width=0),
)

fig.add_shape(
                type="rect",
                x0=0, 
                y0=min(ys)-5,
                x1=max(xs)+5,
                y1=100,
                fillcolor="yellow", opacity=0.3,
                line=dict(width=0),
)

fig.add_annotation(x=min(xs)-3, y=max(ys)+3, text="Reflation", showarrow=False, font_size=15, opacity=0.7)
fig.add_annotation(x=min(xs)-3, y=min(ys)-3, text="Contraction", showarrow=False, font_size=15, opacity=0.7)
fig.add_annotation(x=max(xs)+3, y=max(ys)+3, text="Expansion", showarrow=False, font_size=15, opacity=0.7)
fig.add_annotation(x=max(xs)+3, y=min(ys)-3, text="Reflation", showarrow=False, font_size=15, opacity=0.7)


fig.update_layout(title="Historical Best ETFs",
                  xaxis_title="Strength vs S& 500",
                  yaxis_title="Momentum of Strength",
                  template="plotly_dark",
                  height=700, width=1000,
                  legend_orientation="h",
                  #grid=dict(xgrid=None, ygrid=None)
                  #showlegend=False,
                  hoverlabel=dict(
                        bgcolor="white", 
                        font_size=14, 
                        font_family="Rockwell"
                        ),
                  
                  )
    
fig.show()

In [55]:
lookback = 10

quadrant_df_sub = quadrant_df.iloc[0:lookback]
quadrant_df_sub

,JNK_value1,JNK_value2,JNK_RoC,JNK_RoC_qqe,JNK_gmean,LQD_value1,LQD_value2,LQD_RoC,LQD_RoC_qqe,LQD_gmean,...,RPV_value1,RPV_value2,RPV_RoC,RPV_RoC_qqe,RPV_gmean,XLG_value1,XLG_value2,XLG_RoC,XLG_RoC_qqe,XLG_gmean
2019-12-03,-12.823569,-12.409707,0.222793,91.166946,89.312837,-6.499521,-6.275703,0.912584,96.814048,95.717150,...,0.542598,1.003464,-1.410548,93.334903,94.625817,5.037974,4.750255,0.138202,105.928390,105.240935
2019-12-04,-12.836278,-12.409707,0.418004,93.042573,90.515868,-6.435924,-6.275703,0.589639,97.981984,96.319043,...,0.148077,0.608414,-1.272058,92.162693,93.811728,5.062690,4.776323,0.124271,106.064376,105.814991
2019-12-05,-12.867455,-12.409707,0.400183,94.872696,92.085342,-6.423959,-6.275703,0.267739,98.827099,97.205412,...,-0.218677,0.241432,-0.583059,91.804359,93.005195,5.095738,4.810965,0.048403,105.620492,105.802976
2019-12-06,-13.052652,-12.576311,-0.073941,96.072164,93.770122,-6.564424,-6.275703,-0.243824,99.214827,98.205160,...,-0.535960,-0.076168,-0.495639,91.597540,92.222436,5.164801,4.882082,0.057592,105.290266,105.725459
2019-12-09,-13.105844,-12.629854,0.081538,97.588515,95.379449,-6.633593,-6.275703,-0.113408,99.726819,98.935626,...,-0.764576,-0.305191,-0.245831,91.781572,91.836313,5.174158,4.893882,-0.144538,103.390034,105.086314
2019-12-10,-13.024954,-12.629854,0.135103,99.116618,96.899380,-6.675169,-6.275703,-0.149117,100.162405,99.481502,...,-0.990888,-0.531881,-0.367195,91.788603,91.742980,5.159333,4.893882,-0.078295,102.246354,104.127569
2019-12-11,-12.961431,-12.629854,-0.515640,99.168025,97.978008,-6.690493,-6.282314,-1.472014,98.983599,99.520863,...,-1.243781,-0.785929,0.240589,92.432553,91.899522,5.202455,4.927654,-0.050623,101.454884,103.085344
2019-12-12,-13.030885,-12.629854,-0.824073,98.824675,98.672370,-6.911998,-6.507842,-2.033675,97.474759,99.081604,...,-1.187649,-0.785929,0.908052,94.245432,92.556596,5.139593,4.927654,-0.043782,100.678742,101.937578
2019-12-13,-13.085643,-12.629854,-0.706140,98.679774,98.947066,-6.992163,-6.591724,-1.169896,96.987983,98.394205,...,-1.357052,-0.902194,0.203539,95.016924,93.361699,5.137074,4.927654,0.002933,100.363769,101.183310
2019-12-16,-13.230279,-12.765930,-0.393177,98.913023,98.896215,-7.200030,-6.803000,-1.117210,96.605412,97.508775,...,-1.528600,-1.075863,0.090724,95.593661,94.314579,5.227973,4.962700,0.056177,100.552575,100.761639


In [56]:
def create_etfs_chart():
    fig = go.Figure()

    xs, ys = [], []
    annotations = []

    for ticker in list(ticker_dict.keys()):
        x = quadrant_df_sub[ticker+'_value1'].values.tolist()
        y = quadrant_df_sub[ticker+'_gmean'].values.tolist()
        xs.extend(x)
        ys.extend(y)

        fig.add_trace(go.Scatter(x=x,y=y, 
                                 customdata=[(ticker, ticker_dict[ticker])]*len(x),
                                 hovertemplate="Ticker: %{customdata[0]}<br>"+\
                                               "Name: %{customdata[1]}<br>"+\
                                               "X: %{x: .2f}<br>Y: %{y: .2f}",
                                 name=ticker,
                                 mode='markers',
                                 marker=dict(size=8)))
        
        
        for i in range(len(x)-1):
            if i < 3:
                fig.add_annotation(
                                   x= x[i+1], y= y[i+1], ## X, Y Ends
                                   ax= x[i],  ay= y[i], ## X, Y Starts
                                   xref="x", yref="y",
                                   axref = "x", ayref='y',
                                   text="",
                                   showarrow=True,
                                   arrowhead = 2,
                                   arrowwidth=1.5,)
            else:
                annotations.append(dict(
                                   x= x[i+1], y= y[i+1], ## X, Y Ends
                                   ax= x[i],  ay= y[i], ## X, Y Starts
                                   xref="x", yref="y",
                                   axref = "x", ayref='y',
                                   text="",
                                   showarrow=True,
                                   arrowhead = 2,
                                   arrowwidth=1.5,))
                                #arrowcolor="white",)


    fig.add_shape(
                    type="rect",
                    x0=min(xs)-3, 
                    y0=min(ys)-3,
                    x1=0,
                    y1=100,
                    fillcolor="red", opacity=0.3,
                    line=dict(width=0),
    )

    fig.add_shape(
                    type="rect",
                    x0=0, 
                    y0=100,
                    x1=max(xs)+3,
                    y1=max(ys)+3,
                    fillcolor="green", opacity=0.3,
                    line=dict(width=0),
    )

    fig.add_shape(
                    type="rect",
                    x0=min(xs)-3, 
                    y0=100,
                    x1=0,
                    y1=max(ys)+3,
                    fillcolor="yellow", opacity=0.3,
                    line=dict(width=0),
    )

    fig.add_shape(
                    type="rect",
                    x0=0, 
                    y0=min(ys)-3,
                    x1=max(xs)+3,
                    y1=100,
                    fillcolor="yellow", opacity=0.3,
                    line=dict(width=0),
    )

    fig.add_annotation(x=min(xs)-1.5, y=max(ys)+1.5, text="Reflation", showarrow=False, font_size=15, opacity=0.7)
    fig.add_annotation(x=min(xs)-1.5, y=min(ys)-1.5, text="Contraction", showarrow=False, font_size=15, opacity=0.7)
    fig.add_annotation(x=max(xs)+1.5, y=max(ys)+1.5, text="Expansion", showarrow=False, font_size=15, opacity=0.7)
    fig.add_annotation(x=max(xs)+1.5, y=min(ys)-1.5, text="Reflation", showarrow=False, font_size=15, opacity=0.7)
    
    fig["layout"]["annotations"] += tuple(annotations)

    fig.update_layout(title="Historical Best ETFs",
                      xaxis_title="Strength vs S& 500",
                      yaxis_title="Momentum of Strength",
                      #template="plotly_dark",
                      height=700, width=1000,
                      legend_orientation="h",
                      #grid=dict(xgrid=None, ygrid=None)
                      #showlegend=False,
                      hoverlabel=dict(
                            bgcolor="white", 
                            font_size=14, 
                            font_family="Rockwell"
                            ),
                      )

    return fig

In [57]:
fig = create_etfs_chart()

In [58]:
fig

# Dashboard

In [59]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

In [65]:
colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}

csi_graph = dcc.Graph(figure=create_csi_chart())


header = html.Div(children=[html.H1("William's Financial Dashboard", style={"color":"white"})])
left_pan = html.Div(children=[], className = "two columns")
right_pan = html.Div(children=[csi_graph], id="csi_chart", className = "ten columns")

In [66]:
app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[header, left_pan, right_pan])

In [73]:
def retrieve_csi_main_indexes(url):
    csi_df = None
    try:
        res = requests.get(url)
        if res.status_code == 200:
            csi_df = pd.DataFrame(json.loads(json.loads(res.text)))
            csi_df.index = pd.to_datetime([datetime.datetime.fromtimestamp(int(dt[:-3])) for dt in csi_df.index])
    except Exception as e:
        print(e)

    return csi_df
    
def create_csi_df():
    today = datetime.datetime.today()
    start = today - datetime.timedelta(days=1095)
    today_str = datetime.datetime.strftime(today, "%Y%m%d")        #"20191231" #"%d%d%d"%(today.year, today.month, today.day)
    start_str = datetime.datetime.strftime(start, "%Y%m%d")        #"20180101" # "%d%d%d"%(start.year, start.month, start.day)
    #print("Today's Date : ",today_str)
    #print("Start Date : ",start_str)
    
    url = "http://csitrader.io/csi/parameters?start_date=%s&end_date=%s"%(start_str, today_str)
    
    try:
        csi_df = retrieve_csi_main_indexes(url)
        if not isinstance(csi_df, pd.DataFrame):
            raise Exception("Failed to Retrieve data")
    except Exception as e:
        print("Failed to retrieven CSI Basic Indexes first time. Trying Again.")
        
        try:
            csi_df = retrieve_csi_main_indexes(url)
        except Exception as e:
            print("Failed to retrieve CSI Basic Indexes 2nd Time.")    
    
    csi_df = csi_df.fillna(value=0)
    
    return csi_df, start_str, today_str

def retrieve_quadrant_data(url, ticker):
    ticker_df = None
    try:
        res = requests.get(url)
        if res.status_code == 200:
            ticker_df = pd.DataFrame(json.loads(json.loads(res.text)))
            ticker_df.index = pd.to_datetime([datetime.datetime.fromtimestamp(int(dt[:-3])) for dt in ticker_df.index])
    except Exception as e:
        print("Failed to retreive ticker data")
    
    return ticker_df
    
def create_quadrant_dataframe(ticker_dict):
    res = requests.get("http://csitrader.io/ticker_list")
    if res.status_code == 200:
        tickers = json.loads(res.text)["active_tickers"]
    else:
        tickers = ['JNK', 'LQD', 'CWB', 'ITA', 'HYG', 'PBS', 'PSP', 'PRF', 'IWS', 'RSP', 'XLB', 'IWP', 'IAI', 'IWR', 'SPXL', 'ITOT', 'IHI', 'PRFZ', 'JKE', 'RPV', 'XLG']

    quadrant_df = None

    for ticker in ticker_dict.keys():
        try:
            url = "http://csitrader.io/quadrant/parameters?ticker=%s"%ticker
            ticker_df = retrieve_quadrant_data(url, ticker)
            if not isinstance(ticker_df, pd.DataFrame):
                raise Exception("Ticker %s failed to retrieve quadrant data"%ticker)
                
            quadrant_df = ticker_df.copy() if not isinstance(quadrant_df,pd.DataFrame) else quadrant_df.join(ticker_df)
        except:
            print("Ticker %s failed to retrieve quadrant data. We'll try one more time."%ticker)
            
            try:
                ticker_df = retrieve_quadrant_data(url, ticker)
                quadrant_df = ticker_df.copy() if not isinstance(quadrant_df,pd.DataFrame) else quadrant_df.join(ticker_df)
            except:
                print("Ticker %s failed to retrieve quadrant data.No More try."%ticker)
                

    quadrant_df = quadrant_df.fillna(value=0)

    return quadrant_df, tickers

def retrieve_quadrant_performance(ticker):
    perf_data  = {"SUM_Q1":0, "SUM_Q2":0, "SUM_Q3":0, "SUM_Q4":0}
    try:
        res = requests.get("http://csitrader.io/performance/parameters?ticker=%s&value=SUM"%(ticker))
        if res.status_code == 200:
            perf_data = json.loads(res.text)
    except:
        print("Failed to Retrieve Ticker : %d data. Will try one more time."%ticker)
        
        try:
            res = requests.get("http://csitrader.io/performance/parameters?ticker=%s&value=SUM"%(ticker))
            if res.status_code == 200:
                perf_data = json.loads(res.text)
        except:
            print("Failed to retrieve ticker data one more time. Will not try now")
            
    return perf_data
                
def create_quadrant_bar_data(ticker_dict2):
    quadrant_bar_data  = []
    for ticker in ticker_dict2.keys():
        perf_data = retrieve_quadrant_performance(ticker)
        quadrant_bar_data.append(perf_data) 
       
    quadrant_bar_df = pd.DataFrame(quadrant_bar_data)
    quadrant_bar_df = quadrant_bar_df * 100
    quadrant_bar_df.index = ticker_dict2.keys()

    return quadrant_bar_df
    
    
def retrieve_company_name_from_ticker(ticker):
    company_name = "UNKNOWN"
    try:
        url = "http://csitrader.io/company_name/parameters?ticker=%s"%ticker
        res = requests.get(url)
        if res.status_code == 200:
            company_name_res = json.loads(res.text)
            if isinstance(company_name_res["company_name"], str):
                company_name = str(company_name_res["company_name"])
            else:
                raise Exception("Failed to retrieve company name for ticker : %s"%ticker)
        else:
            raise Exception("Failed to retrieve company name for ticker : %s"%ticker)
    except:
        print("Failed to retrieve company name for ticker : %s. Trying one more time. "%ticker)
        
        try:
            res = requests.get(url)
            company_name_res = json.loads(res.text)
            if isinstance(company_name_res["company_name"], str):
                company_name = str(company_name_res["company_name"])
        except:
            print("Failed to retrieve company name for ticker : %s. Failed 2nd Time."%ticker)
            
    return company_name

def update_quadrant_df(selected_tickers):
    global quadrant_df, ticker_dict    
    
    
    for ticker in selected_tickers:
        col_name = ticker + "_value1"
        if ticker not in ticker_dict:
            company_name = retrieve_company_name_from_ticker(ticker)
            ticker_dict.update({ticker:company_name})
    
        if col_name not in quadrant_df.columns:
            try:
                url = "http://csitrader.io/quadrant/parameters?ticker=%s"%ticker
                ticker_df = retrieve_quadrant_data(url, ticker)
                
                if not isinstance(ticker_df, pd.DataFrame):
                    raise Exception("Ticker %s failed to retrieve quadrant data"%ticker)
                
                quadrant_df = quadrant_df.join(ticker_df)
            except Exception as e:
                print("Ticker %s failed to retrieve quadrant data. We'll now try one more time."%ticker)
                
                try:
                    ticker_df = retrieve_quadrant_data(url, ticker)
                    quadrant_df = quadrant_df.join(ticker_df)
                except:
                    print("Ticker %s failed to retrieve quadrant data one more time."%ticker)


def retrieve_csi_data(url, ticker):
    ticker_df = None
    try:
        res = requests.get(url)
        if res.status_code == 200:
            ticker_df = pd.DataFrame(json.loads(json.loads(res.text)))
            ticker_df.index = pd.to_datetime([datetime.datetime.fromtimestamp(int(dt[:-3])) for dt in ticker_df.index])
            ticker_df = ticker_df.drop(columns=["value2"])
            ticker_df.columns  = [ticker]
    except:
        print("Failed to retrieve data for ticker : %s"%ticker)
    
    return ticker_df
        
def update_csi_df(selected_tickers, start_str, today_str):
    global csi_df
    
    for ticker in selected_tickers:
        if ticker not in csi_df.columns:
            try:
                url = "http://csitrader.io/qqe_ci/parameters?ticker=%s&start_date=%s&end_date=%s"%(ticker, start_str, today_str)
                #print(url)
                ticker_df = retrieve_csi_data(url, ticker)
                
                if not isinstance(ticker_df, pd.DataFrame):
                    raise Exception("Ticker %s failed to retrieve csi data."%ticker)
                    
                csi_df = csi_df.join(ticker_df)
            except Exception as e:
                print(e)
                
                try:
                    ticker_df = retrieve_csi_data(url, ticker)                    
                    csi_df = csi_df.join(ticker_df)
                except Exception as e:
                    print("Ticker %s failed to retrieve csi data last time"%ticker)

def update_quadrant_bar_df(selected_tickers):
    global quadrant_bar_df, ticker_dict2
    
    for ticker in selected_tickers:
        if ticker not in ticker_dict2:
            company_name = retrieve_company_name_from_ticker(ticker)
            ticker_dict2.update({ticker:company_name})
    
        if ticker not in quadrant_bar_df.index:
            quadrant_data = retrieve_quadrant_performance(ticker)
            ticker_df = pd.DataFrame([quadrant_data], index=[ticker])
            ticker_df =  ticker_df *100
            quadrant_bar_df = pd.concat([quadrant_bar_df, ticker_df])

def create_csi_chart(selected_values, start_str, today_str):

    update_csi_df(selected_values, start_str, today_str)
    
    fig = go.Figure()

    charts = []
    for idx, col in enumerate(selected_values):
        if col not in csi_df.columns:
            continue
            
        ticker_industry_name = csi_mapping.get(col.upper()) if csi_mapping.get(col.upper(), None) else col.upper()
        scat =  go.Scatter(x=csi_df.index, y=csi_df[col], name=col.upper(),
                        line = dict(color=color_scales[idx]),
                        hovertemplate="Ticker: %s<br>"%(col.upper())+\
                                      "Name : %s<br>"%(ticker_industry_name)+\
                                      "X: %{x}<br>Y: %{y: .2f}"

                       )
        charts.append(scat)


    fig = go.Figure(data=charts)



    fig.update_layout(title_text="Cyclical Strength Index",
                      xaxis_title='Date Range',
                      yaxis_title='Index',
                      legend_title_text="Ticker",
                      template="plotly_dark",
                      #paper_bgcolor="#21252C", plot_bgcolor="#21252C",
                      autosize=True,
                      margin=dict(t=80, b=50),
                      )

    fig.update_xaxes(rangeslider_visible=True,

                     rangeselector=dict(
                                     buttons=[
                                     dict(count=1, label="1m", step="month", stepmode="backward"),
                                     dict(count=3, label="3m", step="month", stepmode="backward"),
                                     dict(count=6, label="6m", step="month", stepmode="backward"),
                                     dict(count=1, label="YTD", step="year", stepmode="todate"),
                                     dict(count=1, label="1y", step="year", stepmode="backward"),
                                     dict(step="all")
                                    ],
                                     font = dict(size=11.5, color='#000000')
                                )
                    )

    # Set initial "zoom" range of the slider
    zoom_end_dt = datetime.datetime.today()
    zoom_start_dt = datetime.datetime.today() - datetime.timedelta(days=180)

    zoom_end_str = datetime.datetime.strftime(zoom_end_dt, "%Y-%m-%d")
    zoom_start_str = datetime.datetime.strftime(zoom_start_dt, "%Y-%m-%d")

    initial_range = [
        zoom_start_str, zoom_end_str
    ]

    fig['layout']['xaxis'].update(range=initial_range)

    # Change default "hover" effect to "Compare data on hover"
    fig.update_layout(hovermode='x')

    return fig

In [76]:
csi_mapping = {
    'csi1': 'Cyclical Strength Index',
    'csi2': 'Cyclical Strength Index Lag',
    'XLC': 'Communications',
    'XLY': 'Consumer Discretionary',
    'XLP': 'Consumer Staples',
    'XLE': 'Energy',
    'XLF': 'Financials',
    'XLV': 'Healthcare',
    'XLI': 'Industrials',
    'XLB': 'Materials',
    'XLRE': 'Real Estate',
    'XRT': 'Retail',
    'XSD': 'Semiconductors',
    'XLK': 'Technology',
    'XLU': 'Utilities',
    'UUP' : 'Dollar Index',
    'TLT' : 'Long-term Treasuries'
}

colors = {
    'background': '#21252C',
    'text': '#7FDBFF'
}

color_scales = px.colors.cyclical.HSV + px.colors.diverging.Spectral + px.colors.cyclical.mygbm
color_scales = color_scales * 3
rng  = np.random.RandomState(123)
rng.shuffle(color_scales)


ticker_dict = {'ITA' : 'Aerospace & Defense', 'QQQ' : 'Nasdaq 100', 'KO' : 'Coca-Cola Corporation', 'BLK' : 'Blackrock', 'ALL' : 'Allstate'}

ticker_dict2 = {"IWM" :"Russell 200", "SPY":"S&P 500", "LQD":"Corporate Bonds", "TLT":"20+ YR Treasuries"}

#csi_df = pd.read_csv("csi_and_sector_data.csv", index_col=0, parse_dates=True)
csi_df, start_str, today_str = create_csi_df()
#today_str, start_str = "20191231", "20180101"
#pd.read_csv('quadrant_data.csv', index_col=0, parse_dates=True)

quadrant_df, tickers = create_quadrant_dataframe(ticker_dict)

quadrant_bar_df = create_quadrant_bar_data(ticker_dict2)

tickers = tickers + list(ticker_dict2.keys())

slider_value = 8
csi_selects = csi_df.columns[:2]

options_csi = []
for key, val in csi_mapping.items():
    options_csi.append({"label": val, "value": key })

options_etf = []
for ticker in sorted(tickers):
    options_etf.append({"label": ticker, "value": ticker })

# Added "config={"displayModeBar": False}" to hide the displayModeBar from the user. For this graph, I feel that onlyl showing the range slider is sufficient.
csi_graph = dcc.Graph(figure=create_csi_chart(csi_df.columns[:2], start_str, today_str), id="csi_chart", className="figure", style={"height":"70vh",}, config={"displayModeBar": False})

etf_graph = dcc.Graph(figure=None, id="etf_chart", className="figure", style={"height":"65vh",})

performance_bar_graph = dcc.Graph(figure=None, id="performance_bar_chart", className="figure", style={"height":"70vh"})


left_pan_radio = dcc.RadioItems(options = [{'label': 'CSI Chart', 'value': 'CSI'},
                                           {'label': 'ETFs Chart', 'value': 'ETF'},
                                           {'label': '10 YR Performance Chart', 'value': 'PERF'},], value="CSI", id="chart-selection", style={"color":"white", "font-size":20})

radio_div = html.Div(children=[left_pan_radio], style={"display":"flex","justifyContent":"center"})

h1 = html.H1("William's Financial Dashboard", className="title-header", style={"color":"white", "textAlign": "center", "padding":"10px"})
h4 = html.H4("List of Available Charts", style={"color":"white", "textAlign": "center"})

header = html.Div(children=[h1])
header2 = html.Div(children=[h4])

csi_label = html.Label(children="CSI MultiSelect", style={"color":"white", "margin":"3px"})

multi_select_csi = dcc.Dropdown(
                id="multi_select_csi",
                options=options_csi,
                value=[opt["value"] for opt in options_csi][:2],
                multi=True,
                persistence=True, persistence_type="session",
                clearable=False,
                # Adding placeholder text for style purposes
                placeholder = "Add Ticker to Graph",
                style={"background-color": "#21252C", "padding":"2px","margin":"2px 0px 3px 0px", "select-value-color":"red"}
            )

etf_label = html.Label(children="ETFs MultiSelect", style={"color":"white", "margin":"3px"})

multi_select_etf = dcc.Dropdown(
                id="multi_select_etf",
                options=options_etf,
                value=list(ticker_dict.keys()), #[opt["value"] for opt in options_etf],
                multi=True,
                persistence=True, persistence_type="session",
                clearable=False,
                style={"background-color": "#21252C", "padding":"2px","margin":"2px 0px 3px 0px"}
            )

slider_label = html.Label(children="Lookback Window", style={"color":"white", "padding":"3px"})

etf_lookback_slider = dcc.Slider(
                                id="lookback_slider",
                                min=1,
                                max=quadrant_df.shape[0],
                                marks={i: str(i) for i in range(1, quadrant_df.shape[0]+1)},
                                value=8,
                                persistence=True, persistence_type="session",
                                )

toggle_button = daq.ToggleSwitch(
                                id="toggle_markers",
                                label='Markers',
                                labelPosition='top',
                                theme="dark",
                                color="grey",
                                style={"color":"white"},
                                value=True,
                                size=50,
                                persistence=True, persistence_type="session"
                                )

perf_label = html.Label(children="10-YR Performance MultiSelect", style={"color":"white", "margin":"3px"})

multi_select_perf = dcc.Dropdown(
                id="multi_select_perf",
                options=options_etf,
                value=list(ticker_dict2.keys()), #[opt["value"] for opt in options_etf],
                multi=True,
                persistence=True, persistence_type="session",
                clearable=False,
                style={"background-color": "#21252C", "padding":"2px","margin":"2px 0px 3px 0px"}
            )

right_pane_children = html.Div(children=[csi_label, multi_select_csi, csi_graph])
right_pan = html.Div(id="right-pane", children=[right_pane_children], className="nine columns", style={'backgroundColor': colors['background'], "padding":"0px", "margin":"0px"})


left_pan = html.Div(children=[header2, radio_div], className="three columns", style={'backgroundColor': colors['background']})


slider_div = html.Div(children=[slider_label, etf_lookback_slider], className="eleven columns")


main_dashboard = children=html.Div(children=[left_pan, right_pan], className="row")


layout = html.Div(style={'backgroundColor': colors['background'], "height":"120vh"},
                        children=[header, main_dashboard], className="row")

In [85]:
layout.children[1].children[1]

Div(children=[Div([Label(children='CSI MultiSelect', style={'color': 'white', 'margin': '3px'}), Dropdown(id='multi_select_csi', options=[{'label': 'Cyclical Strength Index', 'value': 'csi1'}, {'label': 'Cyclical Strength Index Lag', 'value': 'csi2'}, {'label': 'Communications', 'value': 'XLC'}, {'label': 'Consumer Discretionary', 'value': 'XLY'}, {'label': 'Consumer Staples', 'value': 'XLP'}, {'label': 'Energy', 'value': 'XLE'}, {'label': 'Financials', 'value': 'XLF'}, {'label': 'Healthcare', 'value': 'XLV'}, {'label': 'Industrials', 'value': 'XLI'}, {'label': 'Materials', 'value': 'XLB'}, {'label': 'Real Estate', 'value': 'XLRE'}, {'label': 'Retail', 'value': 'XRT'}, {'label': 'Semiconductors', 'value': 'XSD'}, {'label': 'Technology', 'value': 'XLK'}, {'label': 'Utilities', 'value': 'XLU'}, {'label': 'Dollar Index', 'value': 'UUP'}, {'label': 'Long-term Treasuries', 'value': 'TLT'}], value=['csi1', 'csi2'], clearable=False, multi=True, placeholder='Add Ticker to Graph', style={'backg

In [10]:
start = datetime.datetime.now()
earlier = start - datetime.timedelta(days=3650)

date_range_picker = dcc.DatePickerRange(start_date = earlier, end_date = start)

perf_chart_dropdown = dcc.Dropdown(options = ["SUM", "SQN", "AVG", "STD"], value="SUM", persistence=True, clearable=False,persistence_type="session",)
perf_chart_dropdown

Dropdown(options=['SUM', 'SQN', 'AVG', 'STD'], value='SUM', clearable=False, persistence=True, persistence_type='session')

'2020-05-19'